# Census Data Tools

Initiate and configure the logger. This will save the outputs of the log to the specified file. 

In [1]:
from morpc.logs import config_logs

config_logs('./temp_data/morpc-census-demo.log', level='info') ## Designate the logger level. 

2025-11-10 17:16:33,749 | INFO | morpc.logs.config_logs: Set up logging save to file ./temp_data/morpc-census-demo.log


## morpc.census.api

This module has been developed to fetch and save Census data from various API endpoints. 

## Census API Class

The class CensusAPI class does most of the heavy lifting, formulating the API call, getting the data, wrangling it into a long format, creating frictionless schema and resource files, and saving the data. 

The class takes the following parameters:

1. The "survey/table" to get the data from. 
2. The year of the data. 
3. The group of variables that you want to call.
4. The geographic "scope" of the data you want to get the data for.

Optionally:

5. The geographic scale of the data you want to get the data for.
6. A subset of the variables in the chosen group.



In [2]:
# TODO: Timeseries calls.  

# TODO: Getting select variables from various groups.

## Example: Compare divorced population for all tracts in 15-County region.

### 1. See currently implemented endpoints for "survey/table" parameter.

Use the IMPLEMENTED_ENDPOINTS constant to see currently available options.

In [3]:
from morpc.census.api import IMPLEMENTED_ENDPOINTS

IMPLEMENTED_ENDPOINTS

2025-11-10 17:16:36,821 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data with parameters None.


['acs/acs1',
 'acs/acs1/profile',
 'acs/acs1/subject',
 'acs/acs5',
 'acs/acs5/profile',
 'acs/acs5/subject',
 'dec/pl',
 'dec/dhc',
 'dec/ddhca',
 'dec/ddhcb',
 'dec/sf1',
 'dec/sf2',
 'dec/sf3']

### 2. See available vintages

To see all available endpoints provided by the Census Bureau and their available vintages, see ALL_AVAIL_ENPOINTS

In [4]:
from morpc.census.api import ALL_AVAIL_ENDPOINTS 

ALL_AVAIL_ENDPOINTS['acs/acs5']

[2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023]

### 3. See available variable groups for a survey/table.

In [5]:
from morpc.census.api import get_table_groups

get_table_groups('acs/acs5', 2023).keys()

# TODO: Hierarchical lookup for variables of interest.

2025-11-10 17:16:41,199 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2023/acs/acs5/groups.json with parameters None.


dict_keys(['B01001', 'B01001A', 'B01001B', 'B01001C', 'B01001D', 'B01001E', 'B01001F', 'B01001G', 'B01001H', 'B01001I', 'B01002', 'B01002A', 'B01002B', 'B01002C', 'B01002D', 'B01002E', 'B01002F', 'B01002G', 'B01002H', 'B01002I', 'B01003', 'B02001', 'B02008', 'B02009', 'B02010', 'B02011', 'B02012', 'B02013', 'B02015', 'B02016', 'B02018', 'B02019', 'B02020', 'B02021', 'B03001', 'B03002', 'B03003', 'B04004', 'B04005', 'B04006', 'B04007', 'B05001', 'B05001PR', 'B05002', 'B05003', 'B05003A', 'B05003B', 'B05003C', 'B05003D', 'B05003E', 'B05003F', 'B05003G', 'B05003H', 'B05003I', 'B05004', 'B05005', 'B05005PR', 'B05006', 'B05006PR', 'B05007', 'B05008', 'B05009', 'B05010', 'B05011', 'B05012', 'B05012PR', 'B05013', 'B05014', 'B05015', 'B06001', 'B06001PR', 'B06002', 'B06002PR', 'B06003', 'B06003PR', 'B06004A', 'B06004APR', 'B06004B', 'B06004BPR', 'B06004C', 'B06004CPR', 'B06004D', 'B06004DPR', 'B06004E', 'B06004EPR', 'B06004F', 'B06004FPR', 'B06004G', 'B06004GPR', 'B06004H', 'B06004HPR', 'B0600

###  4. Select the geographic scope of the data.

To see currently implemented scopes. See morpc.census.geos.SCOPES

In [6]:
from morpc.census.geos import SCOPES

SCOPES

# TODO: Passing list of geoids to ucgid parameter in api call

{'us': {'for': 'us:1'},
 'region15': {'in': 'state:39',
  'for': 'county:041,045,049,089,097,129,159,083,101,117,047,073,091,127,141'},
 'region10': {'in': 'state:39',
  'for': 'county:041,045,049,089,097,129,159,083,101,117'},
 'region7': {'in': 'state:39', 'for': 'county:041,045,049,089,097,129,159'},
 'region-corpo': {'in': 'state:39',
  'for': 'county:045,083,097,101,117,129,159'},
 'alabama': {'for': 'state:01'},
 'alaska': {'for': 'state:02'},
 'arizona': {'for': 'state:04'},
 'arkansas': {'for': 'state:05'},
 'california': {'for': 'state:06'},
 'colorado': {'for': 'state:08'},
 'connecticut': {'for': 'state:09'},
 'delaware': {'in': 'state:39', 'for': 'county:041'},
 'district of columbia': {'for': 'state:11'},
 'florida': {'for': 'state:12'},
 'georgia': {'for': 'state:13'},
 'hawaii': {'for': 'state:15'},
 'idaho': {'for': 'state:16'},
 'illinois': {'for': 'state:17'},
 'indiana': {'for': 'state:18'},
 'iowa': {'for': 'state:19'},
 'kansas': {'for': 'state:20'},
 'kentucky': {

### (Optional) 5. Select a smaller geographic scale to get data for. 

For example if you want to get all the tracts in the 15-county MORPC region.

To see available combinations of scope and scale see morpc.census.geos.PSUEDOS.

The key is the parent sumlevel and the values represent the child sumlevel to get the data for.

In [7]:
from morpc.census.geos import PSEUDOS

PSEUDOS

## All available scale levels if the sumlevel of the "scope" paramter is Counties (050)

PSEUDOS['050']

['0600000',
 '06V0000',
 '1000000',
 '1400000',
 '1500000',
 '1600000',
 '7000000',
 '8600000',
 '8710000']

### (Optional) 6. A list of variables to get from the group.

In [8]:
from morpc.census.api import get_group_variables

get_group_variables('acs/acs5', 2023, 'B12001')

2025-11-10 17:16:45,722 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2023/acs/acs5/groups/B12001.json with parameters None.


{'B12001_001E': {'label': 'Estimate!!Total:',
  'concept': 'Sex by Marital Status for the Population 15 Years and Over',
  'predicateType': 'int',
  'group': 'B12001',
  'limit': 0,
  'predicateOnly': True,
  'universe': 'Population 15 years and over'},
 'B12001_001EA': {'label': 'Annotation of Estimate!!Total:',
  'concept': 'Sex by Marital Status for the Population 15 Years and Over',
  'predicateType': 'string',
  'group': 'B12001',
  'limit': 0,
  'predicateOnly': True,
  'universe': 'Population 15 years and over'},
 'B12001_001M': {'label': 'Margin of Error!!Total:',
  'concept': 'Sex by Marital Status for the Population 15 Years and Over',
  'predicateType': 'int',
  'group': 'B12001',
  'limit': 0,
  'predicateOnly': True,
  'universe': 'Population 15 years and over'},
 'B12001_001MA': {'label': 'Annotation of Margin of Error!!Total:',
  'concept': 'Sex by Marital Status for the Population 15 Years and Over',
  'predicateType': 'string',
  'group': 'B12001',
  'limit': 0,
  'pre

In [9]:
variables =['B12001_001E', 'B12001_002E', 'B12001_010E', 'B12001_011E', 'B12001_019E']

### Get the data by calling CensusAPI

In [10]:
from morpc.census.api import CensusAPI

b12001_2023 = CensusAPI('acs/acs5', 2023, 'B12001', 'region15', scale='tract', variables=variables)

2025-11-10 17:16:48,159 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.__init__: Initializing CENSUS_API_DATA object for census-acs-acs5-2023-tract-region15-b12001.
2025-11-10 17:16:48,161 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2023/acs/acs5/groups.json with parameters None.
2025-11-10 17:16:48,793 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.validate: Validating selected parameters
2025-11-10 17:16:48,795 | INFO | morpc.census.api.valid_survey_table: acs/acs5 is valid and implemented.
2025-11-10 17:16:48,795 | INFO | morpc.census.api.valid_vintage: 2023 is valid vintage for acs/acs5
2025-11-10 17:16:48,796 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2023/acs/acs5/groups.json with parameters None.
2025-11-10 17:16:49,406 | INFO | morpc.census.api.valid_group: Group B12001 valid group for 2023 acs/acs5.
2025-11-10 17:16:49,408 | INFO | morpc.

The raw data is available and stored in the class as .DATA

In [14]:
b12001_2023.DATA

,GEO_ID,NAME,B12001_001E,B12001_002E,B12001_010E,B12001_011E,B12001_019E
0,1400000US39041010100,Census Tract 101; Delaware County; Ohio,4670,2370,170,2300,240
1,1400000US39041010200,Census Tract 102; Delaware County; Ohio,5073,2596,385,2477,463
2,1400000US39041010420,Census Tract 104.20; Delaware County; Ohio,2443,1240,144,1203,212
3,1400000US39041010421,Census Tract 104.21; Delaware County; Ohio,2240,1062,81,1178,145
4,1400000US39041010422,Census Tract 104.22; Delaware County; Ohio,4845,2355,44,2490,290
...,...,...,...,...,...,...,...
578,1400000US39159050502,Census Tract 505.02; Union County; Ohio,1959,928,132,1031,179
579,1400000US39159050601,Census Tract 506.01; Union County; Ohio,8524,4337,188,4187,197
580,1400000US39159050602,Census Tract 506.02; Union County; Ohio,3654,1995,156,1659,188
581,1400000US39159050701,Census Tract 507.01; Union County; Ohio,3479,1662,70,1817,72


The class also automatically creates a long table format which is the format we will save the data as. 

In [15]:
b12001_2023.LONG

,GEO_ID,NAME,reference_period,variable_label,variable,estimate
0,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:,B12001_001,4670
3,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:!!Male:,B12001_002,2370
4,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:!!Male:!!Divorced,B12001_010,170
1,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:!!Female:,B12001_011,2300
2,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:!!Female:!!Divorced,B12001_019,240
...,...,...,...,...,...,...
2910,1400000US39159050702,Census Tract 507.02; Union County; Ohio,2023,Total:,B12001_001,2494
2913,1400000US39159050702,Census Tract 507.02; Union County; Ohio,2023,Total:!!Male:,B12001_002,1197
2914,1400000US39159050702,Census Tract 507.02; Union County; Ohio,2023,Total:!!Male:!!Divorced,B12001_010,50
2911,1400000US39159050702,Census Tract 507.02; Union County; Ohio,2023,Total:!!Female:,B12001_011,1297


#### morpc.census.api.DimensionTable

The other class that is implemented is the dimension table. These are typically used for situations in which you want the data to be human readable. 

This is designed to replicate how the Census often represents it's data with the different levels of the variable group as rows and the geography and years as columns. 

In [ ]:
from morpc.census.api import DimensionTable

dim_table = DimensionTable(b12001_2023.LONG)

2025-11-10 17:21:32,298 | INFO | morpc.census.api.DimensionTable.e83a6d0e-6501-4840-9e57-d1770c2bf3b5.__init__: Initializing DIMENSION_TABLE object.


In [18]:
dim_table.wide()

2025-11-10 17:21:34,697 | INFO | morpc.census.api.DimensionTable.e83a6d0e-6501-4840-9e57-d1770c2bf3b5.wide: Pivoting data into wide format.
2025-11-10 17:21:34,699 | INFO | morpc.census.api.DimensionTable.e83a6d0e-6501-4840-9e57-d1770c2bf3b5.create_description_table: Creating description table from variable labels.


GEO_ID                                     1400000US39041010100  \
NAME                    Census Tract 101; Delaware County; Ohio   
reference_period                                           2023   
0      1       2                                                  
Total:                                                     4670   
       Male:                                               2370   
               Divorced                                     170   
       Female:                                             2300   
               Divorced                                     240   

GEO_ID                                     1400000US39041010200  \
NAME                    Census Tract 102; Delaware County; Ohio   
reference_period                                           2023   
0      1       2                                                  
Total:                                                     5073   
       Male:                                               2596   
               Divorced                                     385   
       Female:                                             2477   
               Divorced                                     463   

GEO_ID                                        1400000US39041010420  \
NAME                    Census Tract 104.20; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total:                                                        2443   
       Male:                                                  1240   
               Divorced                                        144   
       Female:                                                1203   
               Divorced                                        212   

GEO_ID                                        1400000US39041010421  \
NAME                    Census Tract 104.21; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total:                                                        2240   
       Male:                                                  1062   
               Divorced                                         81   
       Female:                                                1178   
               Divorced                                        145   

GEO_ID                                        1400000US39041010422  \
NAME                    Census Tract 104.22; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total:                                                        4845   
       Male:                                                  2355   
               Divorced                                         44   
       Female:                                                2490   
               Divorced                                        290   

GEO_ID                                        1400000US39041010520  \
NAME                    Census Tract 105.20; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total:                                                        6081   
       Male:                                                  3188   
               Divorced                                        272   
       Female:                                                2893   
               Divorced                                        313   

GEO_ID                                        1400000US39041010530  \
NAME                    Census Tract 105.30; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                          

Also implemented is the percent table. Which shows the data as a percentage of the total (ie. univierse)

In [19]:
dim_table.percent()

2025-11-10 17:22:35,424 | INFO | morpc.census.api.DimensionTable.e83a6d0e-6501-4840-9e57-d1770c2bf3b5.wide: Pivoting data into wide format.
2025-11-10 17:22:35,426 | INFO | morpc.census.api.DimensionTable.e83a6d0e-6501-4840-9e57-d1770c2bf3b5.create_description_table: Creating description table from variable labels.
2025-11-10 17:22:35,556 | INFO | morpc.census.api.DimensionTable.e83a6d0e-6501-4840-9e57-d1770c2bf3b5.percent: Creating percent table.


GEO_ID                                     1400000US39041010100  \
NAME                    Census Tract 101; Delaware County; Ohio   
reference_period                                           2023   
0      1       2                                                  
Total: Male:                                          50.749465   
               Divorced                                3.640257   
       Female:                                        49.250535   
               Divorced                                5.139186   

GEO_ID                                     1400000US39041010200  \
NAME                    Census Tract 102; Delaware County; Ohio   
reference_period                                           2023   
0      1       2                                                  
Total: Male:                                          51.172876   
               Divorced                                7.589198   
       Female:                                        48.827124   
               Divorced                                9.126749   

GEO_ID                                        1400000US39041010420  \
NAME                    Census Tract 104.20; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total: Male:                                             50.757266   
               Divorced                                   5.894392   
       Female:                                           49.242734   
               Divorced                                   8.677855   

GEO_ID                                        1400000US39041010421  \
NAME                    Census Tract 104.21; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total: Male:                                             47.410714   
               Divorced                                   3.616071   
       Female:                                           52.589286   
               Divorced                                   6.473214   

GEO_ID                                        1400000US39041010422  \
NAME                    Census Tract 104.22; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total: Male:                                             48.606811   
               Divorced                                   0.908153   
       Female:                                           51.393189   
               Divorced                                   5.985552   

GEO_ID                                        1400000US39041010520  \
NAME                    Census Tract 105.20; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total: Male:                                             52.425588   
               Divorced                                   4.472949   
       Female:                                           47.574412   
               Divorced                                   5.147180   

GEO_ID                                        1400000US39041010530  \
NAME                    Census Tract 105.30; Delaware County; Ohio   
reference_period                                              2023   
0      1       2                                                     
Total: Male:                                             48.833765   
               Divorced                                   3.369123   
       Female:                                           51.166235   
               Divorced                                   7.515735   

GEO_ID                                        1400000US39041011101  \
NAME                    Census Tract 111.01; Delawa

### Timeseries of calls

Dimension table takes any long format table that is formatted similar to the output of CensusAPI. 

This allows for easy implementation of timeseries data by concatenating two long form tables from different years.

In [21]:
b12001_2018 = CensusAPI('acs/acs5', 2018, 'B12001', 'region15', scale='tract', variables=variables)

2025-11-10 17:25:11,006 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2018-tract-region15-b12001.__init__: Initializing CENSUS_API_DATA object for census-acs-acs5-2018-tract-region15-b12001.
2025-11-10 17:25:11,008 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2018/acs/acs5/groups.json with parameters None.
2025-11-10 17:25:11,647 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2018-tract-region15-b12001.validate: Validating selected parameters
2025-11-10 17:25:11,648 | INFO | morpc.census.api.valid_survey_table: acs/acs5 is valid and implemented.
2025-11-10 17:25:11,649 | INFO | morpc.census.api.valid_vintage: 2018 is valid vintage for acs/acs5
2025-11-10 17:25:11,649 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2018/acs/acs5/groups.json with parameters None.
2025-11-10 17:25:12,243 | INFO | morpc.census.api.valid_group: Group B12001 valid group for 2018 acs/acs5.
2025-11-10 17:25:12,245 | INFO | morpc.

In [22]:
import pandas as pd

long = pd.concat([b12001_2023.LONG, b12001_2018.LONG])

In [23]:
dim_table = DimensionTable(long)

2025-11-10 17:25:17,667 | INFO | morpc.census.api.DimensionTable.49805c4b-3b83-410d-b2fa-5b0d8be62dc2.__init__: Initializing DIMENSION_TABLE object.


In [28]:
percent = dim_table.percent()

2025-11-10 17:25:48,997 | INFO | morpc.census.api.DimensionTable.49805c4b-3b83-410d-b2fa-5b0d8be62dc2.wide: Pivoting data into wide format.
2025-11-10 17:25:48,999 | INFO | morpc.census.api.DimensionTable.49805c4b-3b83-410d-b2fa-5b0d8be62dc2.create_description_table: Creating description table from variable labels.
2025-11-10 17:25:49,219 | INFO | morpc.census.api.DimensionTable.49805c4b-3b83-410d-b2fa-5b0d8be62dc2.percent: Creating percent table.


In [29]:
percent

GEO_ID                                   1400000US39041010100  \
NAME                  Census Tract 101, Delaware County, Ohio   
reference_period                                         2018   
0     1      2                                                  
Total Male                                          48.549402   
             Divorced                                3.286671   
      Female                                        51.450598   
             Divorced                                6.147292   

GEO_ID                                                         \
NAME                  Census Tract 101; Delaware County; Ohio   
reference_period                                         2023   
0     1      2                                                  
Total Male                                          50.749465   
             Divorced                                3.640257   
      Female                                        49.250535   
             Divorced                                5.139186   

GEO_ID                                   1400000US39041010200  \
NAME                  Census Tract 102, Delaware County, Ohio   
reference_period                                         2018   
0     1      2                                                  
Total Male                                          49.940968   
             Divorced                                5.643447   
      Female                                        50.059032   
             Divorced                                8.642267   

GEO_ID                                                         \
NAME                  Census Tract 102; Delaware County; Ohio   
reference_period                                         2023   
0     1      2                                                  
Total Male                                          51.172876   
             Divorced                                7.589198   
      Female                                        48.827124   
             Divorced                                9.126749   

GEO_ID                                      1400000US39041010420  \
NAME                  Census Tract 104.20, Delaware County, Ohio   
reference_period                                            2018   
0     1      2                                                     
Total Male                                             46.877828   
             Divorced                                   8.190045   
      Female                                           53.122172   
             Divorced                                  11.900452   

GEO_ID                                                            \
NAME                  Census Tract 104.20; Delaware County; Ohio   
reference_period                                            2023   
0     1      2                                                     
Total Male                                             50.757266   
             Divorced                                   5.894392   
      Female                                           49.242734   
             Divorced                                   8.677855   

GEO_ID                                      1400000US39041010421  \
NAME                  Census Tract 104.21, Delaware County, Ohio   
reference_period                                            2018   
0     1      2                                                     
Total Male                                             44.323619   
             Divorced                                   4.915181   
      Female                                           55.676381   
             Divorced                                   7.742497   

GEO_ID                                                            \
NAME                  Census Tract 104.21; Delaware County; Ohio   
reference_period                                            2023   
0     1      2                                                     
Total Mal

### Getting all available datasets in the api.

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [ ]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [ ]:
api = morpc.census.api_get(url, params)

In [ ]:
api

## Geography tools

In [ ]:
from morpc.census import geos

In [ ]:
geos.SCOPES['region15']

In [ ]:
for_params=geos.param_from_scope(scope='region15')


In [ ]:
for_params[1], 

In [ ]:
geos.geoids_from_params(for_params=for_params[1], in_params=for_params[0])

## American Community Survey (ACS) Data Class

When using ACS data, generally we will be digesting data produded using the [morpc-censusacs-fetch](https://github.com/morpc/morpc-censusacs-fetch) workflow. The data that is produced from that script is by default saved in its output_data folders ./morpc-censusacs-fetch/output_data/

The Census ACS Fetch script leverages the `acs_data` class form `morpc.census`


### Create an initial object which represents a variable in the ACS data api.

The class takes 3 arguments:

1. variable group number
2. the year
3. the type of survey (1 or 5 year estimates)

In [ ]:
from morpc.census import ACS

import morpc

morpc.logs.config_logs('./temp_data/morpc-census-demo.log', level='debug')

In [ ]:
acs = ACS('B01001', '2023', 'acs5')

The initial call creates queries the Census for the variable definitions and returns a dictionary of the available variables in the group. see `acs.VARS`

In [ ]:
acs.VARS

The initial call alse fetchs a list of dimensions from a cached json file in ./morpc/census/acs_variable_group.json and is stored in morpc.census.ACS_VAR_GROUPS.

#### Manual verfication for variable dimension names. 

The list of dimensions are automatically created from the Census Variable labels and need verified before being used. If the dimesion names have not be verified, the will not be stored. Navigate to the JSON and check to make sure that there are the correct number of dimension and that they are in the correct order. Change the verfication field to `true`.

In [ ]:
acs.DIMENSIONS

### Query the API for the deisred variables and geography

The `.query()` method queries the API and caches the data in memory under `acs.DATA`. At the same time it creates a frictionless schema that corrosponds with the data. 


In [ ]:
acs = acs.query(for_param='county:*', in_param='state:39')


#### scope:
These are pre-defined sumlevels and scopes for commonly queried geographies. see `morpc.census.SCOPES`.

In [ ]:
acs = acs.scope(scale="block group", scope='region15')

### For custom queries, use for and in parameters to pass to api query. 

#### for_param:
(optional) The geographies for which to call the the query "state:*" represents all states. "state:39" represent Ohio.

#### in_param:
(optional) A filter for the for parameter. In combinations this allows you do call for small geograhpies inside larger ones. 

> Examples: for_param="county:\*", in_param="state:39" would get all counties in Ohio.
> for_param="tract:\*", in_param='state:39,county:041,049' gets all census tracts in Delaware and Franklin Counties.

### Filter the variables using the get parameter

#### get_param:
(Optional) If you want to return a subset of variables, they can be passed here as a list.

### Dimension Tables

When the query is called the class makes table with the dimensions included that can be used to get summaries of the data. 

This can be used to get quick queries for summaries. 

In [ ]:
acs.DIM_TABLE.LONG

In [ ]:
acs.DIM_TABLE.WIDE

In [ ]:
acs.DIM_TABLE.PERCENT

### Save raw data (not dim table) as a frictionless resource with schema

After querying the data, save the data as a frictionless resource with reasonable descriptors. 

In [ ]:
acs.save(output_dir='./temp_data/')

In [ ]:
acs.SCHEMA

In [ ]:
acs.RESOURCE

## Load data from cached file

In [ ]:
import morpc

In [ ]:
acs = morpc.census.ACS('B25010', '2023', '5').load(scope='region15-tracts', dirname='./temp_data/')

## Georeference the data to map

Add geometries by joining GEOS to DATA.

In [ ]:
acs.GEOS

In [ ]:
import geopandas as gpd
acs.DATA = gpd.GeoDataFrame(acs.DATA.join(acs.GEOS), geometry='geometry')

In [ ]:
acs.DATA.plot(column='B01001_002E')

## Use the built in .explore() method to view a map of all the columns in data

In [ ]:
acs.explore(table='PERCENTS')

In [ ]:
acs.MAP

## Using the rest_api module to fetch geometry data from Census API

In [ ]:
import morpc.rest_api as rest_api
import morpc.census as census

In [ ]:

url =  rest_api.get_layer_url(2024, 'county subdivisions', survey='ACS')

query = "STATE = '39' and COUNTY = '049'"

resource = rest_api.resource(
    name = 'morpc-franklin-tracts',
    url = url,
    where = query,
    max_record_count=500
)

In [ ]:
gdf = rest_api.gdf_from_resource(resource)

## Below should still be functional, but hoping to implement into ACS class

#### Load the data using frictionless.load_data()

In [ ]:
data, resource, schema = morpc.frictionless.load_data('./temp_data/morpc-acs5-2023-state-B01001.resource.yaml', verbose=False)

#### Using ACS_ID_FIELDS to get the fields ids

In [ ]:
morpc.census.acs_generate_universe_table(data.set_index("GEO_ID"), "B01001_001")

#### Create a dimension table with the data and the dimension names

In [ ]:
dim_table = morpc.census.acs_generate_dimension_table(data.set_index("GEO_ID"), schema, idFields=idFields, dimensionNames=["Sex", "Age group"])

In [ ]:
dim_table.loc[dim_table['Variable type'] == 'Estimate'].head()

### Build ACS Variable Group JSON for Dimension names